In [74]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


import numpy as np

In [75]:
ALL_GAMES = pd.read_csv('../data/raw/all_games_0630.csv')
NY_GAME = pd.read_csv('~/Downloads/today.csv')

/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_3576/342566199.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_GAMES = pd.read_csv('../data/raw/all_games_0630.csv')


In [76]:
features = ['thrower_x', 'thrower_y', 'start_on_offense', 'point_start_time', 'possession_num', 'possession_throw', 'game_quarter', 'quarter_point', 'is_home_team', 'home_team_score', 'away_team_score', 'gameID', 'total_points', 'home_teamID', 'away_teamID']

def process_games(GAMES):
    GAMES['total_points'] = GAMES['home_team_score'] + GAMES['away_team_score']
    GAMES = GAMES[GAMES.game_quarter < 5] ##TODO include overtimes
    GAMES['point_start_time'] = 720 - GAMES['point_start_time']
    GAMES = GAMES[features]

    games = []
    for gameID in GAMES.gameID.unique():
        GAME = GAMES[GAMES.gameID == gameID]
        points = []
        prev_point_time = 720
        current_quarter = 1
        prev_df = None
        error_points = 0
        for group_keys, group_df in GAME.groupby(['total_points', 'game_quarter']):
            if prev_df is None:
                prev_df = group_df
                continue

            prev_point_time = prev_df.point_start_time.max()
            if (current_quarter != group_df.game_quarter.max()):
                current_quarter = group_df.game_quarter.max()
                current_point_time = 0
            else:
                current_point_time = group_df.point_start_time.max()
            if current_point_time >= prev_point_time:
                error_points = error_points + 1
                continue
            times = np.linspace(prev_point_time, current_point_time + 1, len(prev_df))
            prev_df['times'] = times
            points.append(prev_df)
            prev_df = group_df
            if group_keys == (GAME.home_team_score.max()+GAME.away_team_score.max(), 4):
                times = np.linspace(current_point_time, 0, len(prev_df))
                prev_df['times'] = times
                points.append(prev_df)
        GAME_POINTS = pd.concat(points)
        GAME_POINTS.times = GAME_POINTS.times + ((4 - GAME_POINTS.game_quarter)*720)
        try:
            # if error_points > 0:
            #     print(f'{error_points} errors with {GAME_POINTS.gameID.iloc[0]}')
            assert GAME_POINTS.times.is_monotonic_decreasing, f'timing is off somewhere for game: {GAME_POINTS.gameID.iloc[0]}'
            home_team_win = GAME_POINTS[['away_team_score', 'home_team_score']].max().values.argmax() if GAME_POINTS['home_team_score'].max() != GAME_POINTS['away_team_score'].max() else -2
            GAME_POINTS['home_team_win'] = home_team_win
            games.append(GAME_POINTS)
        except AssertionError as e:
            print(e)

        PROCESSED_GAMES = pd.concat(games).drop(['point_start_time', 'start_on_offense'], axis=1)
        PROCESSED_GAMES['score_diff'] = PROCESSED_GAMES['home_team_score'] - PROCESSED_GAMES['away_team_score']
    return PROCESSED_GAMES[PROCESSED_GAMES.home_team_win >= 0]


In [78]:
df = process_games(ALL_GAMES)

timing is off somewhere for game: 2022-06-17-SEA-OAK


In [80]:
df.to_csv('../data/processed/all_games_0703.csv', index=False)

In [50]:
NY_GAME.gameID.unique()

array(['2023-06-30-DET-PIT', '2023-06-30-SLC-LA', '2023-06-30-COL-POR',
       '2023-06-30-DC-NY'], dtype=object)

In [51]:
NY_GAME[NY_GAME.gameID == '2023-06-30-SLC-LA']

,thrower,thrower_x,thrower_y,receiver,receiver_x,receiver_y,turnover,start_on_offense,point_start_time,current_line,...,quarter_point,point_timeouts,coming_from_recording_team_penalty,coming_from_opposing_team_penalty,is_home_team,home_team_score,away_team_score,gameID,home_teamID,away_teamID
523,jmiller,-11.97,10.64,sconnole,-4.04,31.22,0,True,0,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,0,0,2023-06-30-SLC-LA,aviators,shred
524,sconnole,-4.04,31.22,glindsley,-15.71,44.58,0,True,0,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,0,0,2023-06-30-SLC-LA,aviators,shred
525,glindsley,-15.71,44.58,jmiller,-7.32,38.12,0,True,0,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,0,0,2023-06-30-SLC-LA,aviators,shred
526,jmiller,-7.32,38.12,sconnole,8.02,36.90,0,True,0,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,0,0,2023-06-30-SLC-LA,aviators,shred
527,sconnole,8.02,36.90,wselfridg,10.99,54.32,0,True,0,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,0,0,2023-06-30-SLC-LA,aviators,shred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,jduennebe,-3.00,35.22,wselfridg,-5.00,40.00,0,True,682,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,18,20,2023-06-30-SLC-LA,aviators,shred
1079,wselfridg,-5.00,40.00,sconnole,6.86,34.51,0,True,682,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,18,20,2023-06-30-SLC-LA,aviators,shred
1080,sconnole,6.86,34.51,lyorgason,6.86,34.51,0,True,682,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,18,20,2023-06-30-SLC-LA,aviators,shred
1081,lyorgason,6.86,34.51,jmiller,-1.78,32.90,0,True,682,"['jmiller', 'lyorgason', 'sconnole', 'wselfrid...",...,0,0,0,0,False,18,20,2023-06-30-SLC-LA,aviators,shred
